In [60]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [61]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [62]:
df = pd.read_parquet(os.path.join('data','jizdenky.parquet'))

In [63]:
df = df[df['odjezd'] >= '2024-11-08']

In [64]:
df = df[df['prostredek'] != 'autobus']
df = df[df['predstih_h'] >= 0]

Následující funkce vyfiltruje spojení mezi body A a B a nazpátek nabízené dopravcem C. Parametr "nasobek" určuje, o kolik delší může jízda být oproti nejkratší nalezené.

In [66]:
def filtr(dopravce, mesto1, mesto2, nasobek=1.25):
    dfc = df.copy()
    print(f"{dopravce}: {mesto1}-{mesto2}")
    dfc = dfc[dfc['prodejce'] == dopravce]
    dfc = dfc[dfc['odkud'].str.contains(mesto1) | dfc['kam'].str.contains(mesto1)]
    dfc = dfc[dfc['odkud'].str.contains(mesto2) | dfc['kam'].str.contains(mesto2)]
    minimum_prestupu = dfc['prestupy'].min()
    if minimum_prestupu == -1:
        print("Pozor, vylezlo nám tu -1 přestupů. Opravuji.")
        minimum_prestupu = 0
    print(f"Nejmenší počet přestupů: {minimum_prestupu}.")
    minimum_casu = dfc['jizdni_doba'].min()
    print(f"Nejrychlejší jízdní doba: {minimum_casu} min.")
    dfc = dfc[(dfc['prestupy'] == minimum_prestupu) & (df['jizdni_doba'] <= (minimum_casu * nasobek))]
    dfc = dfc.drop_duplicates(subset=['odjezd','kam','predstih_d'], keep='last')
    print(f"Celkem řádků: {len(dfc)}\n")
    return dfc.reset_index(drop=True)

In [82]:
df.groupby(["prodejce","odkud","kam"]).size().nlargest(100)

prodejce  odkud                          kam                          
ČD        Praha hl.n.                    Brno hl.n.                       49653
          Brno hl.n.                     Praha hl.n.                      47890
          Praha hl.n.                    Ostrava hl.n.                    31473
          Ostrava hl.n.                  Praha hl.n.                      27939
RJ        Brno                           Praha                            20232
          Praha                          Brno                             18349
          Ostrava                        Praha                            11751
ČD        Praha hl.n.                    Bratislava hl.st.                10998
                                         Wien Hbf                          9964
ARR       Liberec                        Pardubice hl.n.                   9568
          Pardubice hl.n.                Liberec                           9547
RJ        Praha                          Ostrava 

In [67]:
cd_berlin = filtr("ČD","Praha hl.n.","Berlin Hbf")
cd_berlin = pd.Series(cd_berlin.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Berlín").head(60)
cd_vars = filtr("ČD","Praha hl.n.","Warszawa Centralna")
cd_vars = pd.Series(cd_vars.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Varšava").head(60)
cd_vid = filtr("ČD","Praha hl.n.","Wien Hbf")
cd_vid = pd.Series(cd_vid.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Vídeň").head(60)
rj_vid = filtr("RJ","Praha","Víde")
rj_vid = pd.Series(rj_vid.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Vídeň").head(60)
cd_brat = filtr("ČD","Praha hl.n.","Bratislava hl.st.")
cd_brat = pd.Series(cd_brat.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Bratislava").head(60)
rj_brat = filtr("RJ","Praha","Bratislava")
rj_brat = pd.Series(rj_brat.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Bratislava").head(60)
rj_cop = filtr('RJ','Praha','Čop')
rj_cop = pd.Series(rj_cop.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Čop").head(60)
cd_brn = filtr('ČD','Praha hl.n.','Bern')
cd_brn = pd.Series(cd_brn.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Bern").head(60)
rj_buda = filtr('RJ','Praha','Budap')
rj_buda = pd.Series(rj_buda.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="RJ Praha-Budapešť").head(60)
cd_buda = filtr('ČD','Praha','Budap')
cd_buda = pd.Series(cd_buda.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="ČD Praha-Budapešť").head(60)
le_krak = filtr("LE","Praha","Krak")
le_krak = pd.Series(le_krak.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="LE Praha-Krakov").head(60)
le_kosi = filtr("LE","Pardub","Košic")
le_kosi = pd.Series(le_kosi.groupby('predstih_d')['cena'].agg(lambda x: [x.max(), x.min(), x.median()]), name="LE Pardubice-Košice").head(60)
ukazat = [cd_vars, cd_berlin, cd_brat, rj_brat, cd_vid, rj_vid, le_kosi, rj_cop, cd_buda, rj_buda, cd_brn, le_krak]
len(ukazat)
for u in ukazat:
    u.name = u.name.replace("-"," &#8596; ")
    u.index = u.index.map(lambda x: f"{x} d")

ČD: Praha hl.n.-Berlin Hbf
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 247.0 min.
Celkem řádků: 6010

ČD: Praha hl.n.-Warszawa Centralna
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 479.0 min.
Celkem řádků: 2168

ČD: Praha hl.n.-Wien Hbf
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 241.0 min.
Celkem řádků: 5532

RJ: Praha-Víde
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 238.0 min.
Celkem řádků: 1591

ČD: Praha hl.n.-Bratislava hl.st.
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 253.0 min.
Celkem řádků: 7560

RJ: Praha-Bratislava
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 250.0 min.
Celkem řádků: 1821

RJ: Praha-Čop
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 645.0 min.
Celkem řádků: 272

ČD: Praha hl.n.-Bern
Nejmenší počet přestupů: 1.
Nejrychlejší jízdní doba: 678.0 min.
Celkem řádků: 325

RJ: Praha-Budap
Nejmenší počet přestupů: 0.
Nejrychlejší jízdní doba: 399.0 min.
Celkem řádků: 2072

ČD: Praha-Budap
Nejmenší poče

Tohle bude úplně odpudivá ad hoc úprava starší funkce pro generování grafů.

In [69]:
def irozhlas_graf(
    carovy=[],
    sloupcovy=[],
    vodorovny=[],
    rozpeti=[],
    procenta=[],
    skryte=[],
    barvy=[
            "#b2e061",  ## světle zelená (light green)
            "#7eb0d5",  ## světle modrá (light blue)
            "#fd7f6f",  ## světle červená (light red)
            "#bd7ebe",  ## světle fialová (light purple)
            "#ffb55a",  ## oranžová (orange)
            "#ffee65",  ## žlutá (yellow)
            "#beb9db",  ## levandulová (lavender)
            "#fdcce5",  ## skoro černá
            "#8bd3c7",  ## světle tyrkysová (light turquoise),
        "red",
        "blue",
        "purple"
        ],
    histogram=False,
    max_procenta=100,
    target="",
    titulek="",
    podtitulek="",
    naproti=[],
    osay=" ",
    osay2=" ",
    osaymin=None,
    osaymax=None,
    kredity=["zdroj dat a autorstvo", "url odkazu"],
    zaokrouhleni=1,
    prvni=True,
    skladany=False,
    naopak=False,
    vzhurunohama=False,
    skrytnuly=False,
):
    """
    Funkce vygeneruje HighCharts graf z pandas Series (jedné nebo více).

    iROZHLAS-friendly barvy:
    - "#b2e061" světle zelená
    - "#7eb0d5" světle modrá
    - "#fd7f6f" světle červená
    - "#bd7ebe" světle fialová
    - "#ffb55a" oranžová
    - "#ffee65" žlutá
    - "#beb9db" levandulová
    - "#fdcce5" skoro černá
    - "#8bd3c7" světle tyrkysová
    """

    import os
    import pandas as pd
    from highcharts_core.chart import Chart
    from highcharts_core.options.series.area import LineSeries
    from highcharts_core.options.series.bar import ColumnSeries
    from highcharts_core.options.series.bar import BarSeries
    from highcharts_core.options.series.area import AreaRangeSeries
    from highcharts_core.options.series.histogram import HistogramSeries
    from highcharts_core.options.legend import Legend
    from highcharts_core.options.title import Title
    from highcharts_core.options.subtitle import Subtitle
    from highcharts_core.options.credits import Credits

    pocitadlo_barev = 0
    
    nastaveni = {}

    if prvni:
        zdrojaky = f"""<script src="https://code.highcharts.com/highcharts.js"></script><script src="https://code.highcharts.com/highcharts-more.js"></script><script src="https://code.highcharts.com/modules/exporting.js"></script><script src="https://code.highcharts.com/modules/export-data.js"></script><script src="https://code.highcharts.com/modules/accessibility.js"></script><style type="text/css">text{{font-family:"Asap"!important}}.paragraph{{font-family:"Noticia text"!important}}.href{{color:#666;fill:#666}}.highcharts-title{{font-family:"Noticia text"!important;font-weight:700!important;text-align:left!important;left:10px!important}}.highcharts-subtitle{{text-align:left!important;font-size:.95rem!important;left:10px!important;font-family:"Asap"!important}}.highcharts-data-labels text{{font-size:.85rem!important}}.highcharts-axis-labels text{{font-size:.85rem!important}}text.highcharts-plot-line-label{{font-size:.85rem!important;fill:#666}}text.highcharts-plot-band-label{{font-size:.85rem!important;fill:#666}}text.highcharts-credits{{font-size:.75rem!important}}.highcharts-tooltip span{{font-family:"Asap"!important}}.axis-label-on-tick{{fill:#aaa;color:#aaa}}.mock-empty-line{{fill:#fff;color:#fff}}</style>"""
    else:
        zdrojaky = ""

    pred = f"""{zdrojaky}
        <figure id="{target}">
        <div id="container"></div>
        </figure>
        <script>"""

    if len(carovy) > 0:
        categories = carovy[0].index.to_list()
    if len(sloupcovy) > 0:
        categories = sloupcovy[0].index.to_list()
    if len(vodorovny) > 0:
        categories = vodorovny[0].index.to_list()
    if len(rozpeti) > 0:
        categories = rozpeti[0].index.to_list()

    categories = [str(x) for x in categories]

    nastaveni["xAxis"] = {"categories": categories, "min": 0}
    nastaveni["yAxis"] = [
        {
            "title": {"text": osay},
            "reversed": vzhurunohama,
            "max": osaymax,
            "min": osaymin,
        }
    ]

    if skladany:
        if len(sloupcovy) > 0:
            nastaveni["plotOptions"] = {"column": {"stacking": "normal"}}
        if len(vodorovny) > 0:
            nastaveni["plotOptions"] = {"bar": {"stacking": "normal"}}
    if histogram:
        nastaveni["plotOptions"] = {
            "column": {
                "pointPadding": 0,
                "borderWidth": 0,
                "groupPadding": 0,
                "shadow": False,
            }
        }

    
    if len(procenta) > 0:
        osa_procent = {
            "title": {"text": osay2},
            "max": max_procenta,
            "min": 0,
            "labels": {"format": "{value} %"},
        }

        if len(procenta) != len(carovy) + len(sloupcovy):
            osa_procent["opposite"] = True
            druha_osa = 1
            nastaveni["yAxis"].append(osa_procent)
            nastaveni["alignTicks"] = False
        if len(procenta) == len(carovy) + len(sloupcovy):
            nastaveni["yAxis"] = [osa_procent]
            druha_osa = 0

    if len(naproti) > 0:
        druha_osa = 1
        druha_osa_y = {
            "title": {"text": osay2},
            "opposite": True,
            "max": naproti[0].max(),
            "min": 0,
        }
        nastaveni["yAxis"].append(druha_osa_y)

    my_chart = Chart(container=target, options=nastaveni)

    procenta = [p.name for p in procenta]
    naproti = [n.name for n in naproti]
    skryte = [s.name for s in skryte]

    def vykresleni(serie, typ):

        pocitadlo_barev = 0
        
        for s in serie:
            popisek = s.name
            print(popisek)

            if s.name in skryte:
                viditelnost = False
            else:
                viditelnost = True

            ktera_osa = 0
            if s.name in naproti:
                ktera_osa = druha_osa

            if s.name in procenta:
                s = [round(x * 100, zaokrouhleni) for x in s.fillna(0).to_list()]
                my_chart.add_series(
                    typ(
                        data=s,
                        visible=viditelnost,
                        name=popisek,
                        y_axis=druha_osa,
                        tooltip={"valueSuffix": " %"},
                    )
                )

            if any(s.name == x.name for x in rozpeti):
                line_id = f"line_{popisek}"
                print(line_id)
                my_chart.add_series(
                    LineSeries(
                        data=s.apply(lambda x: x[2]).fillna(0).to_list(),
                        visible=viditelnost,
                        id = line_id,
                        name=popisek,
                        y_axis=ktera_osa,
                        color = barvy[pocitadlo_barev],
                        tooltip={"valuePrefix": "střední: ","valueSuffix": " Kč"},
                    ))
                my_chart.add_series(
                    AreaRangeSeries(
                        data=s.apply(lambda x: x[0:2]).fillna(0).to_list(),
                        visible=viditelnost,
                        type='arearange',
                        name=popisek,
                        linkedto = line_id,
                        y_axis=ktera_osa,
                        color = barvy[pocitadlo_barev],
                        fillOpacity=0.3,
                        tooltip={"valueSuffix": " Kč"},
                    lineWidth=0,
                    marker={"enabled": False}
                    ))
                pocitadlo_barev += 1

   #         else:
                #my_chart.add_series(
                    #typ(
                     #   data=s.fillna(0).to_list(),
                    #    visible=viditelnost,
                   #     name=popisek,
                  #      y_axis=ktera_osa,
                 #   )
                #)

    if len(sloupcovy) > 0:
        vykresleni(sloupcovy, ColumnSeries)
    if len(carovy) > 0:
        vykresleni(carovy, LineSeries)
    if len(vodorovny) > 0:
        vykresleni(vodorovny, BarSeries)
    if len(rozpeti) > 0:
        vykresleni(rozpeti, AreaRangeSeries)

    my_chart.options.colors = barvy

    if naopak:
        my_chart.options.legend = Legend(reversed=True)

    my_chart.options.title = Title(text=titulek, align="left", margin=30)

    if len(podtitulek) > 0:
        my_chart.options.subtitle = Subtitle(text=podtitulek, align="left")

    my_chart.options.credits = Credits(text=kredity[0], enabled=True, href=kredity[1])

    as_js_literal = my_chart.to_js_literal()

    if skrytnuly == True:
        as_js_literal = as_js_literal.replace("y: 0.0", "y: null")

 #   if len(rozpeti) > 0:
    
  #      as_js_literal = as_js_literal.replace(
   #         '"type":"arearange"', 
    #        '"type":"arearange", "linkedTo": "previous", "fillOpacity": 0.3, "lineWidth": 0, "marker": {"enabled": false}'
     #   )

    
#    if len(rozpeti) > 0:
#        as_js_literal = as_js_literal.replace("type: 'arearange'", "type: 'arearange',linkedTo: 'previous',fillOpacity: 0.3,lineWidth: 0,marker: {enabled: false}")

    as_js_literal = as_js_literal.splitlines()

    as_js_literal2 = []
    
    for line in as_js_literal:
        if 'id: ' in line:
            ajdy = line.split("'")[1]
            print(ajdy)
        if "type: 'arearange'" in line:
            line = line.replace(" type: 'arearange'",f" type: 'arearange',linkedTo: '{ajdy}', fillOpacity: 0.3, lineWidth: 0")
        as_js_literal2.append(line)
    as_js_literal2 = "\n".join(as_js_literal2)
    
    code = f"<html><head><title>{titulek}</title></head><body>{pred}{as_js_literal2}</script></body></html>"

    
    if not os.path.exists("grafy"):
        os.mkdir("grafy")

    with open(os.path.join("grafy", target + ".html"), "w+") as f:
        f.write(code)

    with open(os.path.join("grafy", target + ".txt"), "w+") as f:
        f.write(f"{pred}{as_js_literal2}</script>")

        print("Graf uložen.")

In [80]:
irozhlas_graf(rozpeti=ukazat, skryte=ukazat[1:], target='mezistatni_jizdne', titulek='Mezistátní jízdné podle předstihu nákupu', osaymin=0, podtitulek='Graf zobrazuje nejnižší, střední a nejvyšší cenu. Zahrnuje pouze přímé vlaky anebo spoje s minimálním počtem přestupů na dané trase.', kredity=['Zdroj dat: e-shopy dopravců od 7. 11. do 25. 11. 2024. Vizualizace: iROZHLAS.cz','https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika'])

ČD Praha &#8596; Varšava
line_ČD Praha &#8596; Varšava
ČD Praha &#8596; Berlín
line_ČD Praha &#8596; Berlín
ČD Praha &#8596; Bratislava
line_ČD Praha &#8596; Bratislava
RJ Praha &#8596; Bratislava
line_RJ Praha &#8596; Bratislava
ČD Praha &#8596; Vídeň
line_ČD Praha &#8596; Vídeň
RJ Praha &#8596; Vídeň
line_RJ Praha &#8596; Vídeň
LE Pardubice &#8596; Košice
line_LE Pardubice &#8596; Košice
RJ Praha &#8596; Čop
line_RJ Praha &#8596; Čop
ČD Praha &#8596; Budapešť
line_ČD Praha &#8596; Budapešť
RJ Praha &#8596; Budapešť
line_RJ Praha &#8596; Budapešť
ČD Praha &#8596; Bern
line_ČD Praha &#8596; Bern
LE Praha &#8596; Krakov
line_LE Praha &#8596; Krakov
line_ČD Praha &#8596; Varšava
line_ČD Praha &#8596; Berlín
line_ČD Praha &#8596; Bratislava
line_RJ Praha &#8596; Bratislava
line_ČD Praha &#8596; Vídeň
line_RJ Praha &#8596; Vídeň
line_LE Pardubice &#8596; Košice
line_RJ Praha &#8596; Čop
line_ČD Praha &#8596; Budapešť
line_RJ Praha &#8596; Budapešť
line_ČD Praha &#8596; Bern
line_LE Praha &